In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

plt.style.use('ggplot')

import nltk

df_train = pd.read_csv('/content/playground_train.csv')
df_test = pd.read_csv("/content/playground_test.csv")

In [ ]:
df_train.head()

,id,text,label
0,1,RT Pak Jokowi Pernah bilang : Lahan Pak Bowok ...,Politik
1,2,@V/FLmIFIyENg58ypw0Mtaja5Z6SUNTXlYRxfkm6P5FU= ...,Sosial Budaya
2,3,RT Inilah Kenyataannya...modern War yg harus d...,Pertahanan dan Keamanan
3,4,kiki_daliyo Ganjar Pranowo Ini cara #GanjarMer...,Ideologi
4,5,"Coblos dengan hati nurani, guys. Ganjar Pranow...",Politik


In [ ]:
df_test.head()

,id,text
0,3312,RT Berita terbaru.. kepuasan publik Terhadp jo...
1,3313,RT cara ngeblok swing suara ke anies itu semud...
2,3314,RT RESPECT...!! warga disabilitas diberi kesem...
3,3315,@y3tydTN7WgLsf5TWGJMKloQjMdkYBKdHLW0EhSyIcrk= ...
4,3316,@ImALjYump1bDMeoQfpAlwd5e5iiEuTSEI8EyMN+FCUs= ...


In [ ]:
df_train['label'].value_counts()

,count
label,
Politik,2082
Sosial Budaya,412
Pertahanan dan Keamanan,280
Ideologi,280
Ekonomi,257


In [ ]:
df_train.isnull().sum()
df_test.isnull().sum()

,0
id,0
text,0


## Basic Removal

In [ ]:
import re

# cleaning text
def cleaning_text(text):
    # remove url
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    text =  url_pattern.sub(r'', text)

    # remove hashtags
    # only removing the hash # sign from the word
    text = re.sub(r'#', '', text)

    # remove mention handle user (@)
    text = re.sub(r'@[\w]*', ' ', text)

    # remove punctuation
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    for x in text.lower():
        if x in punctuations:
            text = text.replace(x, " ")

    # remove extra whitespace
    text = text.strip()

    # lowercase
    text = text.lower()
    return text

## Remove Slang Words

In [ ]:
# import Indonesian lexicon
spell = pd.read_csv("https://raw.githubusercontent.com/nasalsabila/kamus-alay/master/colloquial-indonesian-lexicon.csv")

# replace internet slang
def replace_slang(text):
    words = text.split()
    updated_words = [spell.loc[spell['slang'] == word, 'formal'].values[0] if word in spell['slang'].values else word for word in words]
    return " ".join(updated_words)

In [ ]:
import nltk
from nltk.corpus import stopwords
import requests
nltk.download('stopwords')
nltk.download('punkt')

# CONSTRUCT STOPWORDS
rama_stopword = "https://raw.githubusercontent.com/ramaprakoso/analisis-sentimen/master/kamus/stopword.txt"
yutomo_stopword = "https://raw.githubusercontent.com/yasirutomo/python-sentianalysis-id/master/data/feature_list/stopwordsID.txt"
fpmipa_stopword = "https://raw.githubusercontent.com/onlyphantom/elangdev/master/elang/word2vec/utils/stopwords-list/fpmipa-stopwords.txt"
sastrawi_stopword = "https://raw.githubusercontent.com/onlyphantom/elangdev/master/elang/word2vec/utils/stopwords-list/sastrawi-stopwords.txt"
aliakbar_stopword = "https://raw.githubusercontent.com/onlyphantom/elangdev/master/elang/word2vec/utils/stopwords-list/aliakbars-bilp.txt"
pebahasa_stopword = "https://raw.githubusercontent.com/onlyphantom/elangdev/master/elang/word2vec/utils/stopwords-list/pebbie-pebahasa.txt"
elang_stopword = "https://raw.githubusercontent.com/onlyphantom/elangdev/master/elang/word2vec/utils/stopwords-id.txt"
nltk_stopword = stopwords.words('indonesian')

# create path url for each stopword
path_stopwords = [rama_stopword, yutomo_stopword, fpmipa_stopword, sastrawi_stopword,
                  aliakbar_stopword, pebahasa_stopword, elang_stopword]

# combine stopwords
stopwords_l = nltk_stopword
for path in path_stopwords:
    response = requests.get(path)
    stopwords_l += response.text.split('\n')

custom_st = '''
yg yang dgn ane smpai bgt gua gwa si tu ama utk udh btw
ntar lol ttg emg aj aja tll sy sih kalo nya trsa mnrt nih
ma dr ajaa tp akan bs bikin kta pas pdahl bnyak guys abis tnx
bang banget nang mas amat bangettt tjoy hemm haha sllu hrs lanjut
bgtu sbnrnya trjadi bgtu pdhl sm plg skrg
'''

# create dictionary with unique stopword
st_words = set(stopwords_l)
custom_stopword = set(custom_st.split())

# result stopwords
stop_words = st_words | custom_stopword
print(f'Stopwords: {list(stop_words)[:5]}')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Stopwords: ['sebagian', 'mempertanyakan', 'kinilah', 'semisal', 'sekali-kali']


In [ ]:
from nltk import word_tokenize, sent_tokenize

def remove_stopword(text, stop_words=stop_words):
    word_tokens = word_tokenize(text)
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    return ' '.join(filtered_sentence)

In [ ]:
!pip install Sastrawi

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemming_text(text):
  """
  Fungsi untuk melakukan stemming pada teks menggunakan Sastrawi.

  Args:
      text: Teks yang akan disteming.

  Returns:
      Teks yang telah disteming.
  """
  return stemmer.stem(text)


In [ ]:
def preprocess(text):
    # cleaning text and lowercase
    output = cleaning_text(text)

    # remove stopwords
    output = remove_stopword(output)

    # remove slang
    output = replace_slang(output)

    return output

In [ ]:
df_train['clean'] = df_train['text'].map(preprocess)
df_test['clean'] = df_test['text'].map(preprocess)

df_train.head()

,id,text,label,clean
0,1,RT Pak Jokowi Pernah bilang : Lahan Pak Bowok ...,Politik,jokowi bilang lahan bowok kalimantan timur 220...
1,2,@V/FLmIFIyENg58ypw0Mtaja5Z6SUNTXlYRxfkm6P5FU= ...,Sosial Budaya,flmifiyeng58ypw0mtaja5z6suntxlyrxfkm6p5fu= pil...
2,3,RT Inilah Kenyataannya...modern War yg harus d...,Pertahanan dan Keamanan,kenyataannya modern war hadapi negeri paparan ...
3,4,kiki_daliyo Ganjar Pranowo Ini cara #GanjarMer...,Ideologi,kiki daliyo ganjar pranowo ganjarmerawatpancas...
4,5,"Coblos dengan hati nurani, guys. Ganjar Pranow...",Politik,coblos hati nurani ganjar pranowo pilihan ajak...


In [ ]:
df_test.head()

,id,text,clean
0,3312,RT Berita terbaru.. kepuasan publik Terhadp jo...,berita terbaru kepuasan publik terhadp jokowi ...
1,3313,RT cara ngeblok swing suara ke anies itu semud...,ngeblok swing suara anies semudah ngetwit poli...
2,3314,RT RESPECT...!! warga disabilitas diberi kesem...,respect warga disabilitas desak anies re hercu...
3,3315,@y3tydTN7WgLsf5TWGJMKloQjMdkYBKdHLW0EhSyIcrk= ...,= +yxjhoszc6dlue2buc+akx7ianxpidql0pzmcb njps=...
4,3316,@ImALjYump1bDMeoQfpAlwd5e5iiEuTSEI8EyMN+FCUs= ...,+fcus= pendukung prabowo gibran berkat rfg gib...


In [ ]:
# prompt: lakukan fitur ektraksi

from sklearn.feature_extraction.text import TfidfVectorizer

# Inisialisasi TF-IDF Vectorizer
vectorizer = TfidfVectorizer()

# Fit dan transform data training
tfidf_train = vectorizer.fit_transform(df_train['clean'])

# Transform data testing (hanya transform, tidak fit)
tfidf_test = vectorizer.transform(df_test['clean'])

# Melihat bentuk matriks TF-IDF
print(f"Shape of TF-IDF matrix for training data: {tfidf_train.shape}")
print(f"Shape of TF-IDF matrix for testing data: {tfidf_test.shape}")


Shape of TF-IDF matrix for training data: (3311, 12404)
Shape of TF-IDF matrix for testing data: (1415, 12404)


In [ ]:
# Convert the TF-IDF matrix to a DataFrame
tfidf_df_train = pd.DataFrame(tfidf_train.toarray(), columns=vectorizer.get_feature_names_out())

# Optionally, you can also convert the testing data TF-IDF to a DataFrame
tfidf_df_test = pd.DataFrame(tfidf_test.toarray(), columns=vectorizer.get_feature_names_out())

# Now you have the TF-IDF values stored in DataFrames
print(tfidf_df_train.head())
print(tfidf_df_test.head())


    00       000  007koteka   01   02  02bangunindustrikarbon  \
0  0.0  0.581116        0.0  0.0  0.0                     0.0   
1  0.0  0.000000        0.0  0.0  0.0                     0.0   
2  0.0  0.000000        0.0  0.0  0.0                     0.0   
3  0.0  0.000000        0.0  0.0  0.0                     0.0   
4  0.0  0.000000        0.0  0.0  0.0                     0.0   

   02makinsulitdisusul  02melanjutkan  02menang  02menangkanhatirakyat  ...  \
0                  0.0            0.0       0.0                    0.0  ...   
1                  0.0            0.0       0.0                    0.0  ...   
2                  0.0            0.0       0.0                    0.0  ...   
3                  0.0            0.0       0.0                    0.0  ...   
4                  0.0            0.0       0.0                    0.0  ...   

   âƒâœã  âƒã   âˆ  âˆã  ãªâ  ãƒâ   ëœ  ëœã   œã   šã  
0    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
1    0.0  0.0  0.0  0.

In [ ]:
# prompt: lakukan klasifikasi menggunakan rf gunakan dataset test nya menggunakan df_test

import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Memisahkan fitur (X) dan target (y)
X = tfidf_train
y = df_train['label']

# Membagi data menjadi data training dan data validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Inisialisasi model Random Forest
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Melatih model dengan data training
rf_classifier.fit(X_train, y_train)

# Memprediksi label pada data validation
y_pred_val = rf_classifier.predict(X_val)

# Memprediksi label pada data testing
y_pred_test = rf_classifier.predict(tfidf_test)

# Membuat DataFrame untuk hasil prediksi
df_submission = pd.DataFrame({'id': df_test['id'], 'label': y_pred_test})

# Menyimpan hasil prediksi ke file CSV
df_submission.to_csv('submission_rf_colab.csv', index=False)

print("Prediction on test data saved to 'submission_rf.csv'")


Prediction on test data saved to 'submission_rf.csv'


In [ ]:
# prompt: sekarang gunakan logistic regression

import pandas as pd
from sklearn.linear_model import LogisticRegression

# Memisahkan fitur (X) dan target (y)
X = tfidf_train
y = df_train['label']

# Membagi data menjadi data training dan data validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Inisialisasi model Logistic Regression
lr_classifier = LogisticRegression(max_iter=1000)  # Increase max_iter if needed

# Melatih model dengan data training
lr_classifier.fit(X_train, y_train)

# Memprediksi label pada data validation
y_pred_val = lr_classifier.predict(X_val)

# Memprediksi label pada data testing
y_pred_test = lr_classifier.predict(tfidf_test)

# Membuat DataFrame untuk hasil prediksi
df_submission = pd.DataFrame({'id': df_test['id'], 'label': y_pred_test})

# Menyimpan hasil prediksi ke file CSV
df_submission.to_csv('submission_lr_colab.csv', index=False)

print("Prediction on test data saved to 'submission_lr.csv'")


Prediction on test data saved to 'submission_lr.csv'


In [ ]:
# prompt: sekarang gunakan rfc dengan hyperparameter tuning

import pandas as pd
from sklearn.model_selection import GridSearchCV

# Memisahkan fitur (X) dan target (y)
X = tfidf_train
y = df_train['label']

# Membagi data menjadi data training dan data validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Definisikan parameter yang akan di-tune
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Inisialisasi model Random Forest
rf_classifier = RandomForestClassifier(random_state=42)

# Gunakan GridSearchCV untuk mencari hyperparameter terbaik
grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid,
                           cv=5, n_jobs=-1, verbose=2)

# Latih model dengan GridSearchCV
grid_search.fit(X_train, y_train)

# Cetak hyperparameter terbaik yang ditemukan
print("Best parameters found: ", grid_search.best_params_)

# Gunakan model terbaik untuk memprediksi label pada data validation
best_rf_classifier = grid_search.best_estimator_
y_pred_val = best_rf_classifier.predict(X_val)

# Evaluasi performa model pada data validation
print(classification_report(y_val, y_pred_val))

# Memprediksi label pada data testing dengan model terbaik
y_pred_test = best_rf_classifier.predict(tfidf_test)

# Membuat DataFrame untuk hasil prediksi
df_submission = pd.DataFrame({'id': df_test['id'], 'label': y_pred_test})

# Menyimpan hasil prediksi ke file CSV
df_submission.to_csv('submission_rf_tuned_colab.csv', index=False)

print("Prediction on test data saved to 'submission_rf_tuned.csv'")


Fitting 5 folds for each of 81 candidates, totalling 405 fits
Best parameters found:  {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}
                         precision    recall  f1-score   support

                Ekonomi       0.85      0.61      0.71        54
               Ideologi       0.97      0.45      0.62        64
Pertahanan dan Keamanan       0.89      0.60      0.71        52
                Politik       0.75      0.97      0.84       403
          Sosial Budaya       0.88      0.33      0.48        90

               accuracy                           0.78       663
              macro avg       0.87      0.59      0.67       663
           weighted avg       0.81      0.78      0.75       663

Prediction on test data saved to 'submission_rf_tuned.csv'


In [ ]:
# prompt: sekarang gunakan svc

import pandas as pd
from sklearn.svm import SVC

# Memisahkan fitur (X) dan target (y)
X = tfidf_train
y = df_train['label']

# Membagi data menjadi data training dan data validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Inisialisasi model SVC
svc_classifier = SVC(kernel='linear', probability=True)  # Anda dapat mencoba kernel lain seperti 'rbf'

# Melatih model dengan data training
svc_classifier.fit(X_train, y_train)

# Memprediksi label pada data validation
y_pred_val = svc_classifier.predict(X_val)

# Evaluasi performa model pada data validation
print(classification_report(y_val, y_pred_val))

# Memprediksi label pada data testing
y_pred_test = svc_classifier.predict(tfidf_test)

# Membuat DataFrame untuk hasil prediksi
df_submission = pd.DataFrame({'id': df_test['id'], 'label': y_pred_test})

# Menyimpan hasil prediksi ke file CSV
df_submission.to_csv('submission_svc_colab.csv', index=False)

print("Prediction on test data saved to 'submission_svc.csv'")


                         precision    recall  f1-score   support

                Ekonomi       0.86      0.67      0.75        54
               Ideologi       1.00      0.44      0.61        64
Pertahanan dan Keamanan       0.84      0.52      0.64        52
                Politik       0.74      0.97      0.84       403
          Sosial Budaya       0.83      0.32      0.46        90

               accuracy                           0.77       663
              macro avg       0.85      0.58      0.66       663
           weighted avg       0.80      0.77      0.74       663

Prediction on test data saved to 'submission_svc.csv'


In [ ]:
# prompt: sekarang coba SMOTE terlebih dahulu lalu lakukan klasifikasi

import pandas as pd
from imblearn.over_sampling import SMOTE

# Memisahkan fitur (X) dan target (y)
X = tfidf_train
y = df_train['label']

# Membagi data menjadi data training dan data validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Inisialisasi SMOTE
smote = SMOTE(random_state=42)

# Melakukan oversampling pada data training
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Inisialisasi model Random Forest (atau model lainnya)
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Melatih model dengan data training yang telah di-oversampling
rf_classifier.fit(X_train_resampled, y_train_resampled)

# Memprediksi label pada data validation
y_pred_val = rf_classifier.predict(X_val)

# Evaluasi performa model pada data validation
print(classification_report(y_val, y_pred_val))

# Memprediksi label pada data testing
y_pred_test = rf_classifier.predict(tfidf_test)

# Membuat DataFrame untuk hasil prediksi
df_submission = pd.DataFrame({'id': df_test['id'], 'label': y_pred_test})

# Menyimpan hasil prediksi ke file CSV
df_submission.to_csv('submission_rf_smote_colab.csv', index=False)

print("Prediction on test data saved to 'submission_rf_smote.csv'")


                         precision    recall  f1-score   support

                Ekonomi       0.79      0.63      0.70        54
               Ideologi       0.91      0.47      0.62        64
Pertahanan dan Keamanan       0.80      0.62      0.70        52
                Politik       0.76      0.96      0.85       403
          Sosial Budaya       0.85      0.39      0.53        90

               accuracy                           0.78       663
              macro avg       0.82      0.61      0.68       663
           weighted avg       0.79      0.78      0.76       663

Prediction on test data saved to 'submission_rf_smote.csv'


In [ ]:
# prompt: sekarang lakukan SMOTE juga untuk logistic regression

import pandas as pd
# Memisahkan fitur (X) dan target (y)
X = tfidf_train
y = df_train['label']

# Membagi data menjadi data training dan data validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Inisialisasi SMOTE
smote = SMOTE(random_state=42)

# Melakukan oversampling pada data training
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Inisialisasi model Logistic Regression
lr_classifier = LogisticRegression(max_iter=1000)

# Melatih model dengan data training yang telah di-oversampling
lr_classifier.fit(X_train_resampled, y_train_resampled)

# Memprediksi label pada data validation
y_pred_val = lr_classifier.predict(X_val)

# Evaluasi performa model pada data validation
print(classification_report(y_val, y_pred_val))

# Memprediksi label pada data testing
y_pred_test = lr_classifier.predict(tfidf_test)

# Membuat DataFrame untuk hasil prediksi
df_submission = pd.DataFrame({'id': df_test['id'], 'label': y_pred_test})

# Menyimpan hasil prediksi ke file CSV
df_submission.to_csv('submission_lr_smote_colab.csv', index=False)

print("Prediction on test data saved to 'submission_lr_smote.csv'")


                         precision    recall  f1-score   support

                Ekonomi       0.82      0.76      0.79        54
               Ideologi       0.77      0.58      0.66        64
Pertahanan dan Keamanan       0.82      0.77      0.79        52
                Politik       0.82      0.91      0.86       403
          Sosial Budaya       0.70      0.53      0.60        90

               accuracy                           0.80       663
              macro avg       0.78      0.71      0.74       663
           weighted avg       0.80      0.80      0.79       663

Prediction on test data saved to 'submission_lr_smote.csv'


In [ ]:
# prompt: sekarang lakukan SMOTE juga untuk SVM

import pandas as pd
# Memisahkan fitur (X) dan target (y)
X = tfidf_train
y = df_train['label']

# Membagi data menjadi data training dan data validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Inisialisasi SMOTE
smote = SMOTE(random_state=42)

# Melakukan oversampling pada data training
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Inisialisasi model SVC
svc_classifier = SVC(kernel='linear', probability=True)

# Melatih model dengan data training yang telah di-oversampling
svc_classifier.fit(X_train_resampled, y_train_resampled)

# Memprediksi label pada data validation
y_pred_val = svc_classifier.predict(X_val)

# Evaluasi performa model pada data validation
print(classification_report(y_val, y_pred_val))

# Memprediksi label pada data testing
y_pred_test = svc_classifier.predict(tfidf_test)

# Membuat DataFrame untuk hasil prediksi
df_submission = pd.DataFrame({'id': df_test['id'], 'label': y_pred_test})

# Menyimpan hasil prediksi ke file CSV
df_submission.to_csv('submission_svc_smote_colab.csv', index=False)

print("Prediction on test data saved to 'submission_svc_smote.csv'")


                         precision    recall  f1-score   support

                Ekonomi       0.78      0.74      0.76        54
               Ideologi       0.83      0.55      0.66        64
Pertahanan dan Keamanan       0.77      0.58      0.66        52
                Politik       0.78      0.92      0.85       403
          Sosial Budaya       0.71      0.44      0.55        90

               accuracy                           0.78       663
              macro avg       0.78      0.65      0.69       663
           weighted avg       0.78      0.78      0.77       663

Prediction on test data saved to 'submission_svc_smote.csv'


In [ ]:
# prompt: sekarang lakukan SMOTE juga untuk naivebayes

import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from imblearn.over_sampling import SMOTE

# Memisahkan fitur (X) dan target (y)
X = tfidf_train
y = df_train['label']

# Membagi data menjadi data training dan data validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Inisialisasi SMOTE
smote = SMOTE(random_state=42)

# Melakukan oversampling pada data training
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Inisialisasi model Naive Bayes
nb_classifier = MultinomialNB()

# Melatih model dengan data training yang telah di-oversampling
nb_classifier.fit(X_train_resampled, y_train_resampled)

# Memprediksi label pada data validation
y_pred_val = nb_classifier.predict(X_val)

# Evaluasi performa model pada data validation
print(classification_report(y_val, y_pred_val))

# Memprediksi label pada data testing
y_pred_test = nb_classifier.predict(tfidf_test)

# Membuat DataFrame untuk hasil prediksi
df_submission = pd.DataFrame({'id': df_test['id'], 'label': y_pred_test})

# Menyimpan hasil prediksi ke file CSV
df_submission.to_csv('submission_nb_smote_colab.csv', index=False)

print("Prediction on test data saved to 'submission_nb_smote.csv'")


                         precision    recall  f1-score   support

                Ekonomi       0.48      0.81      0.60        54
               Ideologi       0.53      0.70      0.60        64
Pertahanan dan Keamanan       0.51      0.81      0.62        52
                Politik       0.86      0.65      0.74       403
          Sosial Budaya       0.54      0.58      0.56        90

               accuracy                           0.67       663
              macro avg       0.58      0.71      0.63       663
           weighted avg       0.72      0.67      0.68       663

Prediction on test data saved to 'submission_nb_smote.csv'


In [ ]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder



# Memisahkan fitur (X) dan target (y)
X = tfidf_train
y = df_train['label']

# Encode target variable to numerical labels
label_encoder = LabelEncoder()  # Create a LabelEncoder object
y = label_encoder.fit_transform(y)  # Fit and transform the target variable

# Membagi data menjadi data training dan data validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Inisialisasi SMOTE
smote = SMOTE(random_state=42)

# Melakukan oversampling pada data training
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Inisialisasi model XGBoost
xgb_classifier = XGBClassifier()

# Melatih model dengan data training yang telah di-oversampling
xgb_classifier.fit(X_train_resampled, y_train_resampled)

# Memprediksi label pada data validation
y_pred_val = xgb_classifier.predict(X_val)

# Evaluasi performa model pada data validation
print(classification_report(y_val, y_pred_val))

# Memprediksi label pada data testing
y_pred_test = xgb_classifier.predict(tfidf_test)

# Decode numerical predictions back to original labels
y_pred_test = label_encoder.inverse_transform(y_pred_test) # Decode back to original labels

# Membuat DataFrame untuk hasil prediksi
df_submission = pd.DataFrame({'id': df_test['id'], 'label': y_pred_test})

# Menyimpan hasil prediksi ke file CSV
df_submission.to_csv('submission_xgb_smote_colab.csv', index=False)

print("Prediction on test data saved to 'submission_xgb_smote.csv'")

              precision    recall  f1-score   support

           0       0.75      0.74      0.75        54
           1       0.78      0.55      0.64        64
           2       0.76      0.71      0.73        52
           3       0.80      0.88      0.84       403
           4       0.65      0.53      0.59        90

    accuracy                           0.78       663
   macro avg       0.75      0.68      0.71       663
weighted avg       0.77      0.78      0.77       663

Prediction on test data saved to 'submission_xgb_smote.csv'


In [ ]:
# prompt: sekarang terapkan rfc dengan hyperparameter tuning menggunakan smote

import pandas as pd
# Memisahkan fitur (X) dan target (y)
X = tfidf_train
y = df_train['label']

# Membagi data menjadi data training dan data validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Inisialisasi SMOTE
smote = SMOTE(random_state=42)

# Melakukan oversampling pada data training
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Definisikan parameter yang akan di-tune
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Inisialisasi model Random Forest
rf_classifier = RandomForestClassifier(random_state=42)

# Gunakan GridSearchCV untuk mencari hyperparameter terbaik
grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid,
                           cv=5, n_jobs=-1, verbose=2)

# Latih model dengan GridSearchCV dan data yang telah di-oversampling
grid_search.fit(X_train_resampled, y_train_resampled)

# Cetak hyperparameter terbaik yang ditemukan
print("Best parameters found: ", grid_search.best_params_)

# Gunakan model terbaik untuk memprediksi label pada data validation
best_rf_classifier = grid_search.best_estimator_
y_pred_val = best_rf_classifier.predict(X_val)

# Evaluasi performa model pada data validation
print(classification_report(y_val, y_pred_val))

# Memprediksi label pada data testing dengan model terbaik
y_pred_test = best_rf_classifier.predict(tfidf_test)

# Membuat DataFrame untuk hasil prediksi
df_submission = pd.DataFrame({'id': df_test['id'], 'label': y_pred_test})

# Menyimpan hasil prediksi ke file CSV
df_submission.to_csv('submission_rf_tuned_smote_colab.csv', index=False)

print("Prediction on test data saved to 'submission_rf_tuned_smote.csv'")


Fitting 5 folds for each of 81 candidates, totalling 405 fits
Best parameters found:  {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
                         precision    recall  f1-score   support

                Ekonomi       0.81      0.65      0.72        54
               Ideologi       0.91      0.47      0.62        64
Pertahanan dan Keamanan       0.84      0.60      0.70        52
                Politik       0.76      0.96      0.85       403
          Sosial Budaya       0.86      0.40      0.55        90

               accuracy                           0.78       663
              macro avg       0.84      0.61      0.69       663
           weighted avg       0.80      0.78      0.76       663

Prediction on test data saved to 'submission_rf_tuned_smote.csv'
